---
## Desafio de projeto - Tradutor de artigos - Azure

O conteúdo a seguir corresponde aos desafio de projeto que realizei durante o bootcamp Bootcamp Microsoft Certification Challenge #1 - AI 102, oferecido pela Dio.

**Observação:**

Alguns dados sensíveis foram retirados do código, portanto para que o tradutor de artigos funcione conforme o esperado, será necessário que você complete com suas próprias informações. Não se preocupe, os comentários no código te nortearão no processo!


**Contatos:**

*   [LinkedIn](https://www.linkedin.com/in/leonardor99/)
*   [GitHub](https://github.com/ldemmz/)
*   [E-mail](mailto:leonardor99@outlook.com)
---


In [ ]:
!pip install requests python-docx
!pip install requests beautifulsoup4 openai langchain-openai
import requests
import os
from docx import Document
from bs4 import BeautifulSoup
from langchain_openai.chat_models.azure import AzureChatOpenAI

In [ ]:
## Dados sensíveis. Complete com as informações de seu serviço de tradução hospedado na Azure.
subscription_key = ""
endpoint = ""
location = ""
target_language = ""
#######################

# Função para tradução de texto
def translator_text(text, target_language):
  path = "/translate"
  constructed_url = endpoint + path

  headers = {
      "Ocp-Apim-Subscription-Key": subscription_key,
      "Ocp-Apim-Subscription-Region": location,
      "Content-type": "application/json",
      "X-ClientTraceId": str(os.urandom(16))
  }

  body = [{"text": text}]

  params = {
      "api-version": "3.0",
      "from": "en",
      "to": [target_language]
  }

  request = requests.post(constructed_url, params=params, headers=headers, json=body)
  response = request.json()
  return response[0]["translations"][0]["text"]

In [ ]:
# Faça o teste
translator_text("One of the key impacts of effective corporate governance is the enhancement of transparency and accountability within organizations. Companies that adopt strong governance practices are better positioned to foster trust among investors, customers, and employees.", target_language)

In [ ]:

# Função para tradução de documento
def translate_document(path):
  document = Document(path)
  full_text = []

  for paragraph in document.paragraphs:
    translated_text = translator_text(paragraph.text, target_language)
    full_text.append(translated_text)

  translated_document = Document()

  for line in full_text:
    translated_document.add_paragraph(line)

  path_translated = path.replace(".docx", f"_{target_language}.docx")
  translated_document.save(path_translated)

  return path_translated

input_file = ""  # Insira o caminho do arquivo desejado
translate_document(input_file)

In [ ]:
# Leitura do arquivo traduzido
def read_translated_document(path):
  document = Document(path)

  for paragraph in document.paragraphs:
    print(paragraph.text)

translated_file = "" # Insira o caminho do arquivo traduzido que fora criado
#Teste
read_translated_document(translated_file)

In [ ]:
# Webscrapping com limpeza de texto
def extract_text_from_url(url):
  response = requests.get(url)

  if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    for script_or_style in soup(["script","style"]):
      script_or_style.decompose()
    text = soup.get_text(separator = " ")
    lines = (line.strip() for line in text.splitlines())
    parts = (phrase.strip() for line in lines for phrase in line.split("  "))
    cleaned_text = '\n'.join(part for part in parts if part)
    return cleaned_text
  else:
    print(f"Failed to fetch the URL. Status code: {response.status_code}")
    return None

extract_text_from_url(" ") # Insira a url desejada

In [ ]:
## Dados sensíveis OpenAI Azure. Você precisa completar com as informações de seu serviço assinado.
client = AzureChatOpenAI(
    azure_endpoint = "",
    api_key = "",
    api_version = "",
    deployment_name = "",
    max_retries=0
)
###########################

# Função para tradução de artigos e conversão em markdown
def translate_article(text, lang):
  messages = [
      ("system", "Você atua como tradutor de textos"),
      ("user", f"Traduza o {text} para o idioma {lang} e responda em markdown")
  ]

  response = client.invoke(messages)
  return response.content

In [ ]:
# Testes de tradução

translate_article("Now it’s time to write some code. Open up your project in Visual Studio Code and create a couple of data structure classes: IrisData.cs and IrisPrediction.cs.", "pt-br")

url = "" # Insira a url desejada para testar
text = extract_text_from_url(url)
article = translate_article(text,"pt-br")

print(article)
